# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#!unzip "/content/drive/MyDrive/UFRPE/6º Período/Redes Neurais/Projeto/projeto_redes_neurais.zip" -d /content/
!unzip "/content/drive/MyDrive/RN/projeto_redes_neurais.zip" -d /content/

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/BoundedQueue.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/ArgvContext.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/ForcePublic.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Condition.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Closure.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/BlockingQueue.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Thread.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/includ

# Imports

In [ ]:
# !pip install -r /content/projeto_redes_neurais/requirements.txt
!pip install -r /content/requirements.txt
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.7 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.0 MB/s eta 0:00:00


In [ ]:
# Bibliotecas padrão
import os
os.environ['TORCHDYNAMO_DISABLE'] = '1'
import sys
import re
import argparse

# Bibliotecas de terceiros
import importlib.metadata
from tqdm import tqdm

# PyTorch e módulos relacionados
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, Subset, DataLoader, random_split

# Torchaudio e torchvision
import torchvision
import torchaudio
import torchaudio.transforms as transforms

# Avaliação de ASR
import jiwer
import numpy as np

from collections import defaultdict
from jiwer import wer
from torch.serialization import add_safe_globals

## Verificar se está usando a GPU

In [ ]:
# Verificar se a GPU está disponível
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device}")

Usando o dispositivo: cuda


## Verificar as dependências

In [ ]:
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Torchaudio version:", torchaudio.__version__)
print("Jiwer version:", importlib.metadata.version("jiwer"))

Torch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
Torchaudio version: 2.5.1+cu124
Jiwer version: 3.1.0


# Dataset

## Pre-processamento da base

1. Baixe a base de teste

In [ ]:
!wget -P /content/projeto_redes_neurais/data/raw https://www.openslr.org/resources/12/test-clean.tar.gz

--2025-02-26 03:13:47--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2025-02-26 03:13:47--  https://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘/content/projeto_redes_neurais/data/raw/test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  21.3MB/s    in 17s     

2025-02-26 03:14:05 (19.5 MB/s) - ‘/content/projeto_redes_neurais/data/raw/test-clean.tar.gz’ saved [346663984/346663984]



2. Extraia a base dentro da pasta do projeto

In [ ]:
!tar -xvzf /content/projeto_redes_neurais/data/raw/test-clean.tar.gz -C /content/projeto_redes_neurais/data/raw/

LibriSpeech/LICENSE.TXT
LibriSpeech/README.TXT
LibriSpeech/CHAPTERS.TXT
LibriSpeech/SPEAKERS.TXT
LibriSpeech/BOOKS.TXT
LibriSpeech/test-clean/
LibriSpeech/test-clean/6930/
LibriSpeech/test-clean/6930/75918/
LibriSpeech/test-clean/6930/75918/6930-75918-0000.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0007.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0006.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0005.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0004.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0002.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0011.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0015.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0003.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0010.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0001.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0018.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0008.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0017.flac
LibriSpeech/test-clean/

In [ ]:
dataset = torchaudio.datasets.LIBRISPEECH(root="/content/projeto_redes_neurais/data/raw/", url="test-clean", download=False)

print(f"Total de amostras: {len(dataset)}")

# Pegando os valores corretos do primeiro áudio
waveform, sample_rate, utterance, _, _, _ = dataset[0]

print(f"Formato do áudio: {waveform.shape}, Taxa de amostragem: {sample_rate}")
print(f"Transcrição: {utterance}")


Total de amostras: 2620
Formato do áudio: torch.Size([1, 166960]), Taxa de amostragem: 16000
Transcrição: HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE


3. Realize o pré-processamento da base de dados e construa o vocabulário

In [ ]:
DATA_DIR = "/content/projeto_redes_neurais/data/raw"
PROCESSED_DIR = "/content/projeto_redes_neurais/data/processed"
MODEL_DIR = "/content/drive/MyDrive/RN/modelos"
DATASET_DIR = "/content/projeto_redes_neurais/datasets"

class LibriSpeechDataset(Dataset):
    def __init__(self, root_dir):
        self.dataset = torchaudio.datasets.LIBRISPEECH(root=root_dir, url="test-clean", download=True)
        self.transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_mels=128,
            n_fft=2048,
            hop_length=512,
            normalized=True
        )
        self.vocab = self._build_vocab()  # Método adicionado

    def _build_vocab(self):
        """Constrói o vocabulário a partir das transcrições"""
        counter = defaultdict(int)
        for idx in range(len(self.dataset)):
            _, _, transcript, _, _, _ = self.dataset[idx]
            for char in transcript.lower():
                counter[char] += 1

        # Caracteres especiais
        vocab = {
            '<blank>': 0,  # Para CTC Loss
            '<unk>': 1      # Caracteres desconhecidos
        }

        # Adiciona caracteres ordenados por frequência
        for idx, (char, count) in enumerate(sorted(counter.items(), key=lambda x: x[1], reverse=True)):
            vocab[char] = idx + 2  # +2 para reservar os primeiros índices
        torch.save(vocab, 'vocab.pt')
        torch.save(vocab, '/content/drive/MyDrive/UFRPE/RN/splits/vocab.pt')
        return vocab

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sample_rate, transcript, _, _, _ = self.dataset[idx]
        waveform = waveform.squeeze(0)  # Remove dimensão extra (1, samples) -> (samples)
        spectrogram = self.transform(waveform)
        spectrogram = spectrogram.unsqueeze(0)  # Adiciona canal (1, n_mels, time)

        # Convert texto para índices
        transcript_indices = [
            self.vocab.get(char, self.vocab['<unk>'])
            for char in transcript.lower()
        ]

        return spectrogram, torch.tensor(transcript_indices, dtype=torch.long)

# Função de pré-processamento corrigida
def preprocess_data():
    os.makedirs(PROCESSED_DIR, exist_ok=True)

    dataset = LibriSpeechDataset(DATA_DIR)

    # Salvar dados processados
    processed_data = {
        'waveforms': [d[0] for d in dataset],
        'transcripts': [d[1] for d in dataset],
        'vocab': dataset.vocab
    }

    torch.save(
        processed_data,
        os.path.join(PROCESSED_DIR, "librispeech_processed.pt"),
        _use_new_zipfile_serialization=True
    )
    print(f"Pré-processamento concluído! {len(dataset)} amostras salvas.")

if __name__ == "__main__":
    preprocess_data()


Pré-processamento concluído! 2620 amostras salvas.


In [ ]:
# Etapa 1: Permitir explicitamente a classe customizada
add_safe_globals([LibriSpeechDataset])

## Carregando Dataset

In [ ]:
# Função collate_fn
# ========== FUNÇÕES ESSENCIAIS ========== #
def collate_fn(batch):
    batch.sort(key=lambda x: x[0].shape[-1], reverse=True)
    waveforms, transcripts = zip(*batch)

    # Padding dos espectrogramas
    max_time = max(w.shape[-1] for w in waveforms)
    waveforms_padded = torch.stack([
        torch.nn.functional.pad(w, (0, max_time - w.shape[-1]))
        for w in waveforms
    ]).squeeze(1).permute(0, 2, 1)  # Resultado: (B, T, F)

    # Processamento dos textos
    transcript_lengths = torch.tensor([len(t) for t in transcripts], dtype=torch.long)
    transcripts_padded = torch.nn.utils.rnn.pad_sequence(transcripts, batch_first=True)

    return waveforms_padded, transcripts_padded, transcript_lengths

# Função para salvar os conjuntos
def save_splits(train_set, val_set, test_set, vocab):
    os.makedirs(DATASET_DIR, exist_ok=True)

    torch.save({
        'train_indices': train_set.indices,  # Correção: Vírgula apenas entre chaves
        'val_indices': val_set.indices,
        'test_indices': test_set.indices,
        'vocab': vocab
    }, os.path.join(DATASET_DIR, "splits.pt"))


# Função para carregar os conjuntos
def load_splits(dataset):
    splits = torch.load(
        os.path.join(DATASET_DIR, "splits.pt"),
        weights_only=True,
        map_location='cpu'
    )

    from torch.utils.data import Subset
    return (
        Subset(dataset, splits['train_indices']),
        Subset(dataset, splits['val_indices']),
        Subset(dataset, splits['test_indices']),
        splits['vocab']
    )

# Arquiteturas

In [ ]:
# Classe do modelo atualizada
class SpeechLSTM(nn.Module):
    def __init__(self, input_size=128, hidden_size=256, num_layers=3, num_classes=30):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return nn.functional.log_softmax(self.fc(out), dim=-1)

class OptimizedSpeechLSTM(nn.Module):
    def __init__(self, input_size=128, hidden_size=512, num_layers=4,
                 num_classes=30, conv_channels=64, dropout=0.3):
        super().__init__()

        # Camada de convolução para extração de features
        self.conv = nn.Sequential(
            nn.Conv2d(1, conv_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(conv_channels),
            nn.Hardswish(),
            nn.MaxPool2d(2),
            nn.Dropout(dropout/2)
        )

        # Ajuste da dimensão para LSTM
        self.adaptative_pool = nn.AdaptiveAvgPool2d((None, hidden_size//2))

        # LSTM Profunda Bidirecional
        self.lstm = nn.LSTM(
            input_size=hidden_size//2,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Camadas de atenção
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size*2,
            num_heads=4,
            dropout=dropout
        )

        # Camadas fully connected
        self.fc = nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )

        # Inicialização de pesos
        self._init_weights()

    def _init_weights(self):
        for name, module in self.named_modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight_ih' in name:
                        nn.init.xavier_normal_(param)
                    elif 'weight_hh' in name:
                        nn.init.orthogonal_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)

    def forward(self, x):
        # Input shape: (B, T, F)
        x = x.unsqueeze(1)  # Add channel dim

        # Conv Feature Extraction
        x = self.conv(x)  # (B, C, T, F)
        x = self.adaptative_pool(x)

        # Reshape for LSTM
        B, C, T, F = x.size()
        x = x.permute(0, 2, 1, 3).reshape(B, T, -1)  # (B, T, C*F)

        # LSTM Layers
        x, _ = self.lstm(x)

        # Attention
        x = x.permute(1, 0, 2)  # (T, B, F) for attention
        x, _ = self.attention(x, x, x)
        x = x.permute(1, 0, 2)  # (B, T, F)

        # Output Layer
        x = self.fc(x)
        return nn.functional.log_softmax(x, dim=-1)

class HybridCTCLSTM(nn.Module):
    def __init__(self, input_size=128, hidden_size=256, num_layers=2,
                 num_classes=30, dropout=0.3):
        super().__init__()
        # --- CNN para extrair features mantendo a dimensão temporal ---
        # Nossa ideia: tratar a dimensão de frequência como "altura" e a de tempo como "largura".
        # Suponha que a entrada seja (B, T, F) com F = 128.
        # Primeiro, transponha para (B, F, T) e insira um canal.
        # Aplicamos convoluções que reduzem somente a dimensão de frequência.
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # Pooling apenas na dimensão de frequência (F), preservando o tempo (T)
            nn.MaxPool2d(kernel_size=(2,1))
        )
        # Após a CNN, a dimensão de frequência é reduzida pela metade: (F//2)
        # E o número de canais é 32. Assim, para cada time-step teremos um vetor de dimensão: 32 * (input_size//2)
        cnn_output_size = 32 * (input_size // 2)
        # Projeção para ajustar à dimensão de entrada do LSTM:
        self.fc_proj = nn.Linear(cnn_output_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        # --- LSTM bidirecional ---
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        # --- Camada final: gera logits para cada time-step ---
        self.classifier = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # x: (B, T, F)  -- onde F corresponde à dimensão de frequência (ex.: 128)
        # Transpor para tratar F como "altura" e T como "largura":
        # Primeiro, transpor para (B, F, T)
        x = x.transpose(1, 2)  # (B, F, T)
        # Adicionar dimensão de canal: (B, 1, F, T)
        x = x.unsqueeze(1)
        # Aplicar a CNN: o pooling (kernel=(2,1)) reduz somente a dimensão de F
        x = self.cnn(x)  # (B, 32, F//2, T)
        # Rearranjar para o LSTM: queremos uma sequência de time-steps com features por time-step.
        # Permutamos para (B, T, 32, F//2)
        x = x.permute(0, 3, 1, 2)
        B, T, C, F_new = x.size()
        # Achatar as dimensões dos canais e frequência:
        x = x.reshape(B, T, C * F_new)  # (B, T, 32*(input_size//2))
        # Projetar para o tamanho de entrada do LSTM:
        x = self.fc_proj(x)  # (B, T, hidden_size)
        x = self.dropout(x)
        # LSTM bidirecional:
        x, _ = self.lstm(x)  # (B, T, hidden_size*2)
        # Classificação: gerar logits para cada time-step
        x = self.classifier(x)  # (B, T, num_classes)
        return F.log_softmax(x, dim=-1)


# Treinamento

## Função evaluate para medir o desempenho no conjunto de validação

In [ ]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets, target_lengths in loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)

            # CORREÇÃO APLICADA AQUI:
            input_lengths = torch.full(
                size=(inputs.size(0),),  # Tupla correta
                fill_value=inputs.size(1),
                dtype=torch.long,
                device=DEVICE
            )

            loss = criterion(
                outputs.permute(1, 0, 2),
                targets.to(DEVICE),
                input_lengths,
                target_lengths.to(DEVICE)
            )
            total_loss += loss.item()

    return total_loss / len(loader)

## Treinamento do modelo Speech Base da LSTM

In [ ]:
# Definir hiperparâmetros
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função de treinamento
def train(model, train_loader, val_loader, criterion, optimizer, vocab):
    best_loss = float('inf')

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        for inputs, targets, target_lengths in train_loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            target_lengths = target_lengths.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)

            input_lengths = torch.full(
                size=(inputs.size(0),),
                fill_value=inputs.size(1),
                dtype=torch.long,
                device=DEVICE
            )

            # Ajuste para formato CTC: (T, N, C)
            log_probs = outputs.permute(1, 0, 2)

            loss = criterion(
                log_probs,
                targets,
                input_lengths,
                target_lengths.to(DEVICE)
            )

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validação
        val_loss = evaluate(model, val_loader, criterion)

        # Salva o melhor modelo
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'vocab': vocab
            }, os.path.join(MODEL_DIR, "best_model_SpeechLSTM.pth"))

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")

# Função principal atualizada
def main():
    # Carregar dados com segurança
    processed_data = torch.load(
        os.path.join(PROCESSED_DIR, "librispeech_processed.pt"),
        weights_only=True,
        map_location='cpu'
    )

    # Reconstruir dataset manualmente
    class TempDataset(Dataset):
      def __init__(self, data):
          self.waveforms = data['waveforms']
          self.transcripts = data['transcripts']
          self.vocab = data['vocab']

      def __len__(self):
          return len(self.waveforms)

      def __getitem__(self, idx):
          waveform = self.waveforms[idx]
          if waveform.dim() == 2:
              waveform = waveform.unsqueeze(0)
          return waveform, self.transcripts[idx]

    full_dataset = TempDataset(processed_data)

    # Divisão e salvamento
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size

    train_set, val_set, test_set = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # [2] Salvar as divisões primeiro
    save_splits(train_set, val_set, test_set, full_dataset.vocab)

    # [3] Agora carregar as divisões salvas
    train_set, val_set, test_set, vocab = load_splits(full_dataset)

    # Cria DataLoaders
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    # Inicializa modelo
    model = SpeechLSTM(num_classes=len(vocab)).to(DEVICE)
    criterion = nn.CTCLoss(blank=vocab['<blank>']).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Treino
    train(model, train_loader, val_loader, criterion, optimizer, vocab)

if __name__ == "__main__":
    os.environ['TORCHDYNAMO_DISABLE'] = '1'
    main()

KeyboardInterrupt: 

## Avalia o modelo de acordo com a métrica WER no conjunto de testes

In [ ]:
# ========== CONFIGURAÇÕES ========== #
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
DATA_DIR = "/content/projeto_redes_neurais/data/raw"
PROCESSED_DIR = "/content/projeto_redes_neurais/data/processed"
MODEL_DIR = "/content/drive/MyDrive/RN/modelos"
DATASET_DIR = "/content/projeto_redes_neurais/datasets"
MODEL_PATH = "/content/drive/MyDrive/RN/modelos/best_model_SpeechLSTM.pth"

# ========== FUNÇÕES ESSENCIAIS ========== #
def collate_fn(batch):
    batch.sort(key=lambda x: x[0].shape[-1], reverse=True)
    waveforms, transcripts = zip(*batch)

    # Padding dos espectrogramas
    max_time = max(w.shape[-1] for w in waveforms)
    waveforms_padded = torch.stack([
        torch.nn.functional.pad(w, (0, max_time - w.shape[-1]))
        for w in waveforms
    ]).squeeze(1).permute(0, 2, 1)

    # Processamento dos textos
    transcript_lengths = torch.tensor([len(t) for t in transcripts], dtype=torch.long)
    transcripts_padded = torch.nn.utils.rnn.pad_sequence(transcripts, batch_first=True)

    return waveforms_padded, transcripts_padded, transcript_lengths

# ========== CARREGAMENTO DE DADOS ========== #
# 1. Carregar dados processados
processed_data = torch.load(
    os.path.join(PROCESSED_DIR, "librispeech_processed.pt"),
    map_location='cpu',
    weights_only=True
)

# 2. Dataset de inferência
class InferenceDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.waveforms = data['waveforms']
        self.transcripts = data['transcripts']
        self.vocab = data['vocab']

    def __len__(self):
        return len(self.waveforms)

    def __getitem__(self, idx):
        return self.waveforms[idx], self.transcripts[idx]

full_dataset = InferenceDataset(processed_data)

# 3. Carregar splits
splits = torch.load(
    os.path.join(DATASET_DIR, "splits.pt"),
    map_location='cpu',
    weights_only=True
)
test_set = Subset(full_dataset, splits['test_indices'])

# ========== CARREGAMENTO DO MODELO ========== #
# 4. Carregar checkpoint
checkpoint = torch.load(
    MODEL_PATH,
    map_location=DEVICE,
    weights_only=True
)

# Extrair vocabulário do checkpoint
if 'vocab' not in checkpoint:
    raise KeyError("Vocabulário não encontrado no checkpoint")

vocab = checkpoint['vocab']

# ========== FUNÇÕES DE DECODIFICAÇÃO ========== #
def decode_predictions(ctc_output, vocab, blank_index=0):
    _, max_indices = torch.max(ctc_output, dim=2)
    decoded_texts = []

    for batch in max_indices:
        current_text = []
        prev_char = -1

        for char_idx in batch:
            char_idx = char_idx.item()
            if char_idx != prev_char and char_idx != blank_index:
                if char_idx in vocab.values():
                    current_text.append(char_idx)
            prev_char = char_idx

        idx_to_char = {v: k for k, v in vocab.items()}
        decoded = ''.join([idx_to_char.get(idx, '') for idx in current_text])
        decoded_texts.append(decoded)

    return decoded_texts

# ========== CÁLCULO DO WER ========== #
def calculate_test_wer():
    test_loader = DataLoader(
        test_set,
        batch_size=BATCH_SIZE,
        collate_fn=collate_fn
    )

    total_wer = 0
    total_samples = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets, target_lengths) in enumerate(test_loader):
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)

            # Decodificar previsões
            predictions = decode_predictions(outputs, vocab)

            # Processar referências, ignorando tokens "<blank>"
            idx_to_char = {v: k for k, v in vocab.items()}
            references = []
            valid_indices = []

            for i, target in enumerate(targets):
                try:
                    # Converter índices para texto, ignorando tokens "<blank>"
                    text = ''.join([idx_to_char[idx.item()]
                                    for idx in target
                                    if idx.item() in idx_to_char and idx_to_char[idx.item()] != '<blank>'])
                    if not text.strip():
                        raise ValueError("Texto vazio")
                    references.append(text)
                    valid_indices.append(i)
                except Exception as e:
                    print(f"Batch {batch_idx}, Amostra {i} ignorada: {str(e)}")
                    continue

            # Para visualização: imprimir a frase real e a prevista para cada amostra válida
            valid_predictions = [predictions[i] for i in valid_indices]
            for ref, pred in zip(references, valid_predictions):
                print(f"Real: {ref}  |  Previsto: {pred}")

            # Calcular WER para o batch
            if len(references) > 0:
                try:
                    batch_wer = wer(references, valid_predictions)
                    total_wer += batch_wer * len(references)
                    total_samples += len(references)
                except Exception as e:
                    print(f"Erro no batch {batch_idx}: {str(e)}")

    return total_wer / total_samples if total_samples > 0 else float('inf')

# ========== EXECUÇÃO PRINCIPAL ========== #
if __name__ == "__main__":
    # Instanciar modelo
    model = SpeechLSTM(num_classes=len(vocab)).to(DEVICE)

    # Carregar pesos
    try:
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Modelo carregado com sucesso!")
    except Exception as e:
        print(f"Falha ao carregar modelo: {str(e)}")
        raise

    model.eval()

    # Calcular WER
    try:
        test_wer = calculate_test_wer()
        print(f"\n✅ Resultado Final - WER no Teste: {test_wer:.4f} ({test_wer*100:.2f}%)")

    except Exception as e:
        print(f"\n❌ Erro na avaliação: {str(e)}")


## Treina o modelo Speech Otimizado da LSTM

In [ ]:
# ========== CONFIGURAÇÕES ========== #
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.001

DATA_DIR = "/content/projeto_redes_neurais/data/raw"
PROCESSED_DIR = "/content/projeto_redes_neurais/data/processed"
MODEL_DIR = "/content/drive/MyDrive/RN/modelos"
DATASET_DIR = "/content/projeto_redes_neurais/datasets"
MODEL_PATH = os.path.join(MODEL_DIR, "best_model_OptimizedSpeechLSTM.pth")

# ========== FUNÇÕES DE TREINAMENTO ========== #
def train(model, train_loader, val_loader, criterion, optimizer, vocab):
    best_loss = float('inf')

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        for inputs, targets, target_lengths in train_loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            target_lengths = target_lengths.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)

            input_lengths = torch.full(
                size=(inputs.size(0),),
                fill_value=outputs.size(1),
                dtype=torch.long,
                device=DEVICE
            )

            log_probs = outputs.permute(1, 0, 2)
            loss = criterion(log_probs, targets, input_lengths, target_lengths)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_loss = evaluate(model, val_loader, criterion)

        # Salva o melhor modelo
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'vocab': vocab
            }, MODEL_PATH)
            print(f"✅ Novo melhor modelo salvo em epoch {epoch+1}")

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")

# ========== FUNÇÃO DE AVALIAÇÃO (VALIDAÇÃO) ========== #
def evaluate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets, target_lengths in val_loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            target_lengths = target_lengths.to(DEVICE)

            outputs = model(inputs)
            input_lengths = torch.full(
                size=(inputs.size(0),),
                fill_value=outputs.size(1),
                dtype=torch.long,
                device=DEVICE
            )

            log_probs = outputs.permute(1, 0, 2)
            loss = criterion(log_probs, targets, input_lengths, target_lengths)
            total_loss += loss.item()

    return total_loss / len(val_loader)

# ========== EXECUÇÃO DO TREINAMENTO ========== #
def main():
    from torch.utils.data import Dataset

    # Carregar dados processados
    processed_data = torch.load(
        os.path.join(PROCESSED_DIR, "librispeech_processed.pt"),
        map_location='cpu'
    )

    # Criar dataset temporário
    class TempDataset(Dataset):
        def __init__(self, data):
            self.waveforms = data['waveforms']
            self.transcripts = data['transcripts']
            self.vocab = data['vocab']

        def __len__(self):
            return len(self.waveforms)

        def __getitem__(self, idx):
            waveform = self.waveforms[idx]
            if waveform.dim() == 2:
                waveform = waveform.unsqueeze(0)
            return waveform, self.transcripts[idx]

    full_dataset = TempDataset(processed_data)

    # Dividir dataset
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_set, val_set, _ = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    # Carregar modelo
    model = OptimizedSpeechLSTM(num_classes=len(full_dataset.vocab)).to(DEVICE)
    criterion = nn.CTCLoss(blank=full_dataset.vocab['<blank>']).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Iniciar treinamento
    train(model, train_loader, val_loader, criterion, optimizer, full_dataset.vocab)

if __name__ == "__main__":
    os.environ['TORCHDYNAMO_DISABLE'] = '1'
    main()


## Avalia o modelo de acordo com a métrica WER no conjunto de testes

In [ ]:
# ========== CONFIGURAÇÕES ========== #
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
MODEL_PATH = os.path.join("/content/drive/MyDrive/RN/modelos", "best_model_OptimizedSpeechLSTM.pth")
PROCESSED_DIR = "/content/projeto_redes_neurais/data/processed"
DATASET_DIR = "/content/projeto_redes_neurais/datasets"

# ========== FUNÇÕES DE DECODIFICAÇÃO ========== #
def decode_predictions(ctc_output, vocab, blank_index=0):
    _, max_indices = torch.max(ctc_output, dim=2)
    decoded_texts = []
    for batch in max_indices:
        current_text = []
        prev_char = -1
        for char_idx in batch:
            char_idx = char_idx.item()
            if char_idx != prev_char and char_idx != blank_index:
                if char_idx in vocab.values():
                    current_text.append(char_idx)
            prev_char = char_idx
        idx_to_char = {v: k for k, v in vocab.items()}
        decoded = ''.join([idx_to_char.get(idx, '') for idx in current_text])
        decoded_texts.append(decoded)
    return decoded_texts

# ========== CÁLCULO DO WER ========== #
def calculate_test_wer(model, test_set, vocab):
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    total_wer = 0
    total_samples = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets, target_lengths) in enumerate(test_loader):
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            predictions = decode_predictions(outputs, vocab)
            idx_to_char = {v: k for k, v in vocab.items()}
            references = [''.join(idx_to_char[idx.item()] for idx in target if idx.item() in idx_to_char and idx_to_char[idx.item()] != '<blank>') for target in targets]
            batch_wer = wer(references, predictions)
            total_wer += batch_wer * len(references)
            total_samples += len(references)
    return total_wer / total_samples if total_samples > 0 else float('inf')

# ========== EXECUÇÃO DA AVALIAÇÃO ========== #
def evaluate_model():
    # Carregar modelo treinado
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    vocab = checkpoint['vocab']

    model = OptimizedSpeechLSTM(num_classes=len(vocab)).to(DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Carregar conjunto de testes
    processed_data = torch.load(os.path.join(PROCESSED_DIR, "librispeech_processed.pt"), map_location='cpu')
    test_set = Subset(processed_data, range(int(0.85 * len(processed_data)), len(processed_data)))

    # Calcular WER
    test_wer = calculate_test_wer(model, test_set, vocab)
    print(f"\n✅ Resultado Final - WER no Teste: {test_wer:.4f} ({test_wer*100:.2f}%)")

if __name__ == "__main__":
    evaluate_model()


## Treinamento do modelo Híbrido da LSTM

In [ ]:
# ========== CONFIGURAÇÕES ========== #
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.001

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = "/content/projeto_redes_neurais/data/raw"
PROCESSED_DIR = "/content/projeto_redes_neurais/data/processed"
MODEL_DIR = "/content/drive/MyDrive/RN/modelos"
DATASET_DIR = "/content/drive/MyDrive/RN/datasets"

# ========== FUNÇÃO DE TREINO ========== #
def train(model, train_loader, val_loader, criterion, optimizer, vocab):
    best_loss = float('inf')  # Inicializa com um valor alto
    best_model_path = os.path.join(MODEL_DIR, "best_model.pth")

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for inputs, targets, target_lengths in train_loader:
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)
            target_lengths = target_lengths.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)

            # Preparar os comprimentos das entradas para o CTCLoss
            input_lengths = torch.full(
                size=(inputs.size(0),),
                fill_value=outputs.size(1),
                dtype=torch.long,
                device=DEVICE
            )

            log_probs = outputs.permute(1, 0, 2)  # (T, N, C)
            loss = criterion(log_probs, targets, input_lengths, target_lengths)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Calcular validação
        val_loss = evaluate(model, val_loader, criterion)

        # Se a perda de validação for menor, salvar o modelo
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'vocab': vocab
            }, best_model_path)
            print(f"✅ Melhor modelo atualizado! Salvo em {best_model_path} (Val Loss: {val_loss:.4f})")

        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")

# ========== EXECUÇÃO PRINCIPAL ========== #
if __name__ == "__main__":
    # Carregar dados processados
    processed_data = torch.load(os.path.join(PROCESSED_DIR, "librispeech_processed.pt"), map_location='cpu')

    # Criar dataset
    class TempDataset(torch.utils.data.Dataset):
        def __init__(self, data):
            self.waveforms = data['waveforms']
            self.transcripts = data['transcripts']
            self.vocab = data['vocab']
        def __len__(self):
            return len(self.waveforms)
        def __getitem__(self, idx):
            return self.waveforms[idx], self.transcripts[idx]

    full_dataset = TempDataset(processed_data)

    # Divisão do dataset
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_set, val_set, _ = random_split(full_dataset, [train_size, val_size, test_size],
                                         generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    # Inicializar modelo e otimizador
    model = HybridCTCLSTM(input_size=128, hidden_size=256, num_layers=2, num_classes=30, dropout=0.3).to(DEVICE)
    criterion = nn.CTCLoss(blank=full_dataset.vocab['<blank>']).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Treinar o modelo
    train(model, train_loader, val_loader, criterion, optimizer, full_dataset.vocab)


Epoch 1/50 | Train Loss: 3.9083 | Val Loss: 3.0324
Epoch 2/50 | Train Loss: 2.9210 | Val Loss: 2.9075
Epoch 3/50 | Train Loss: 2.8935 | Val Loss: 2.8941
Epoch 4/50 | Train Loss: 2.8857 | Val Loss: 2.8829
Epoch 5/50 | Train Loss: 2.8505 | Val Loss: 2.8711
Epoch 6/50 | Train Loss: 2.7979 | Val Loss: 2.8245
Epoch 7/50 | Train Loss: 2.6352 | Val Loss: 7.7569
Epoch 8/50 | Train Loss: 2.4335 | Val Loss: 3.9022
Epoch 9/50 | Train Loss: 2.3015 | Val Loss: 2.3099
Epoch 10/50 | Train Loss: 2.1820 | Val Loss: 2.3470
Epoch 11/50 | Train Loss: 2.0778 | Val Loss: 2.8189
Epoch 12/50 | Train Loss: 1.9844 | Val Loss: 2.0505
Epoch 13/50 | Train Loss: 1.9057 | Val Loss: 2.2662
Epoch 14/50 | Train Loss: 1.8188 | Val Loss: 22.7326
Epoch 15/50 | Train Loss: 1.7450 | Val Loss: 2.9046
Epoch 16/50 | Train Loss: 1.6783 | Val Loss: 2.0295
Epoch 17/50 | Train Loss: 1.6053 | Val Loss: 2.6273
Epoch 18/50 | Train Loss: 1.5446 | Val Loss: 1.8624
Epoch 19/50 | Train Loss: 1.4888 | Val Loss: 1.7167
Epoch 20/50 | Train 

## Avalia o modelo de acordo com a métrica WER no conjunto de testes

In [ ]:
# ========== CONFIGURAÇÕES ========== #
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATASET_DIR = "/content/drive/MyDrive/RN/datasets"
MODEL_DIR = "/content/drive/MyDrive/RN/modelos"
MODEL_PATH = os.path.join(MODEL_DIR, "best_model.pth")

# ========== FUNÇÃO PARA AVALIAÇÃO ========= #
def calculate_test_wer(model, test_loader, vocab):
    total_wer = 0
    total_samples = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets, target_lengths) in enumerate(test_loader):
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            predictions = decode_predictions(outputs, vocab)
            idx_to_char = {v: k for k, v in vocab.items()}
            references = []
            valid_indices = []
            for i, target in enumerate(targets):
                try:
                    text = ''.join([idx_to_char[idx.item()]
                                    for idx in target
                                    if idx.item() in idx_to_char and idx_to_char[idx.item()] != '<blank>'])
                    if not text.strip():
                        raise ValueError("Texto vazio")
                    references.append(text)
                    valid_indices.append(i)
                except Exception as e:
                    print(f"Batch {batch_idx}, Amostra {i} ignorada: {str(e)}")
                    continue
            valid_predictions = [predictions[i] for i in valid_indices]
            for ref, pred in zip(references, valid_predictions):
                print(f"Real: {ref}  |  Previsto: {pred}")
            if len(references) > 0:
                batch_wer = wer(references, valid_predictions)
                total_wer += batch_wer * len(references)
                total_samples += len(references)
    return total_wer / total_samples if total_samples > 0 else float('inf')

# ========== EXECUÇÃO PRINCIPAL ========== #
if __name__ == "__main__":
    # Carregar dataset de teste
    test_set = torch.load(os.path.join(DATASET_DIR, "test_set.pt"), map_location='cpu')

    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    # Carregar modelo treinado
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model = HybridCTCLSTM(input_size=128, hidden_size=256, num_layers=2, num_classes=30, dropout=0.3).to(DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    vocab = checkpoint['vocab']

    # Avaliar o modelo
    test_wer = calculate_test_wer(model, test_loader, vocab)
    print(f"\n✅ Resultado Final - WER no Teste: {test_wer:.4f} ({test_wer*100:.2f}%)")
